# Data Cleaning, Data Merging

## Merged data output available with last date, on Github


The process will be made as .py file afterwards

===================================================

---

# Data Cleaning 1: Population dataset

ACS, 2018

In [1]:
import pandas as pd
import numpy as np
import re
import os
import datetime as dt

In [89]:
# read in population to merge it with output data
pop = pd.read_csv('https://raw.githubusercontent.com/mramadhito/QMSS-KPMG-4/master/us_state_population.csv', dtype = str)
pop['Population'] = pop['Population'].str.replace(',','')
pop['Population'] = pop['Population'].astype(float)
pop = pop.rename(columns={"Region": "StateName", "Population": "population"})
pop.head(10)

,StateName,population
0,United States,328239523.0
1,Northeast,55982803.0
2,Midwest,68329004.0
3,South,125580448.0
4,West,78347268.0
5,Alabama,4903185.0
6,Alaska,731545.0
7,Arizona,7278717.0
8,Arkansas,3017804.0
9,California,39512223.0


---

# Data Cleaning 2: Oxford Policy Dataset (US ONLY)

https://raw.githubusercontent.com/OxCGRT/USA-covid-policy/master/data/OxCGRT_US_latest.csv

Dataset information:
* Aggregated by state
* Reported daily

Columns:
57 columns in total

23
C1_School closing,C1_Flag,C1_Notes,
C2_Workplace closing,C2_Flag,C2_Notes,
C3_Cancel public events,C3_Flag,C3_Notes,
C4_Restrictions on gatherings,C4_Flag,C4_Notes,
C5_Close public transport,C5_Flag,C5_Notes,
C6_Stay at home requirements,C6_Flag,C6_Notes,
C7_Restrictions on internal movement,C7_Flag,C7_Notes,
C8_International travel controls,C8_Notes,


9
E1_Income support,E1_Flag,E1_Notes,
E2_Debt/contract relief,E2_Notes,
E3_Fiscal measures,E3_Notes,
E4_International support,E4_Notes,


11
H1_Public information campaigns,H1_Flag,H1_Notes,
H2_Testing policy,H2_Notes,
H3_Contact tracing,H3_Notes,
H4_Emergency investment in healthcare,H4_Notes,
H5_Investment in vaccines,H5_Notes,

2
M1_Wildcard,M1_Notes,

12
ConfirmedCases,ConfirmedDeaths,
StringencyIndex,StringencyIndexForDisplay,
StringencyLegacyIndex,StringencyLegacyIndexForDisplay,
GovernmentResponseIndex,GovernmentResponseIndexForDisplay,
ContainmentHealthIndex,ContainmentHealthIndexForDisplay,
EconomicSupportIndex,EconomicSupportIndexForDisplay

In [5]:
# Read in the data
oxford_raw = pd.read_csv('https://raw.githubusercontent.com/OxCGRT/USA-covid-policy/master/data/OxCGRT_US_latest.csv')

C:\Users\Garda\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (37) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
oxford_raw.head()

,CountryName,CountryCode,RegionName,RegionCode,Jurisdiction,Date,C1_School closing,C1_Flag,C1_Notes,C2_Workplace closing,...,StringencyIndex,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay
0,United States,USA,NaN,NaN,NAT_GOV,20200101,0.0,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,United States,USA,NaN,NaN,NAT_GOV,20200102,0.0,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,United States,USA,NaN,NaN,NAT_GOV,20200103,0.0,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,United States,USA,NaN,NaN,NAT_GOV,20200104,0.0,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,United States,USA,NaN,NaN,NAT_GOV,20200105,0.0,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# Drop columns not necessary
# only need two columns

oxford = oxford_raw[['RegionName', 'RegionCode', 'Jurisdiction', 'Date', 'StringencyIndexForDisplay', 'ContainmentHealthIndexForDisplay']]
oxford.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17914 entries, 0 to 17913
Data columns (total 6 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   RegionName                        17576 non-null  object 
 1   RegionCode                        17576 non-null  object 
 2   Jurisdiction                      17914 non-null  object 
 3   Date                              17914 non-null  int64  
 4   StringencyIndexForDisplay         17847 non-null  float64
 5   ContainmentHealthIndexForDisplay  17847 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 839.8+ KB


In [8]:
# data transformation
# state only
# non-null states
oxford = oxford[(oxford['Jurisdiction']!='NAT_GOV') & (oxford['RegionName'].notnull())]
oxford['Date'] = oxford['Date'].apply(lambda x: dt.datetime.strptime(str(x), '%Y%m%d'))
oxford['RegionCode'] = [re.sub('US_','', i) for i in oxford['RegionCode']]
oxford = oxford.drop('Jurisdiction', axis=1)
oxford = oxford.rename(columns={'StringencyIndexForDisplay': 'stringency', 'ContainmentHealthIndexForDisplay': 'containment'})

In [9]:
oxford.tail()

,RegionName,RegionCode,Date,stringency,containment
17909,Wyoming,WY,2020-11-29,42.59,52.78
17910,Wyoming,WY,2020-11-30,42.59,52.78
17911,Wyoming,WY,2020-12-01,42.59,52.78
17912,Wyoming,WY,2020-12-02,42.59,52.78
17913,Wyoming,WY,2020-12-03,42.59,52.78


In [10]:
# sanity check: end of March in NY - everything should be closed
oxford[(oxford['Date']=='2020-03-31')&(oxford['RegionName']=='New York')]

,RegionName,RegionCode,Date,stringency,containment
11920,New York,NY,2020-03-31,79.63,66.67


In [11]:
# sanity check: Most recent in NY - everything should be closed
oxford[(oxford['Date']==max(oxford['Date']))&(oxford['RegionName']=='New York')]

,RegionName,RegionCode,Date,stringency,containment
12167,New York,NY,2020-12-03,69.91,75.35


---

# Data Cleaning 3: JHU COVID-19 US Dataset

Data cleaning of JHU COVID-19 USA dataset

Aggregated by state

* Confirmed = cumulative confirmed
* Deaths = cumulative deaths
* Date
* new_confirmed = new confirmed cases each day
* new_deaths = new death cases each day
* new_confirmed_7 = rollling average, 7 days
* new_deaths_7 = rolling average, 7 days
* new_confirmed_14 = rolling average, 14 days
* new_deaths_14 = rolling average, 14 days

In [12]:
raw_cases_us = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv")
raw_deaths_us = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv")

In [15]:
raw_cases_us.tail()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20,12/2/20
3335,84056039,US,USA,840,56039.0,Teton,Wyoming,US,43.935225,-110.589080,...,1433,1442,1476,1476,1547,1570,1606,1644,1655,1678
3336,84056041,US,USA,840,56041.0,Uinta,Wyoming,US,41.287818,-110.547578,...,950,982,1016,1016,1037,1037,1046,1070,1089,1121
3337,84090056,US,USA,840,90056.0,Unassigned,Wyoming,US,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
3338,84056043,US,USA,840,56043.0,Washakie,Wyoming,US,43.904516,-107.680187,...,325,339,395,395,415,417,428,441,468,487
3339,84056045,US,USA,840,56045.0,Weston,Wyoming,US,43.839612,-104.567488,...,372,383,386,386,396,396,396,397,409,415


In [16]:
raw_deaths_us.tail()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20,12/2/20
3335,84056039,US,USA,840,56039.0,Teton,Wyoming,US,43.935225,-110.589080,...,2,2,2,2,2,2,2,2,2,2
3336,84056041,US,USA,840,56041.0,Uinta,Wyoming,US,41.287818,-110.547578,...,4,4,4,4,4,4,4,4,4,4
3337,84090056,US,USA,840,90056.0,Unassigned,Wyoming,US,0.000000,0.000000,...,0,0,0,0,0,0,0,0,9,9
3338,84056043,US,USA,840,56043.0,Washakie,Wyoming,US,43.904516,-107.680187,...,7,7,8,8,8,8,8,8,8,8
3339,84056045,US,USA,840,56045.0,Weston,Wyoming,US,43.839612,-104.567488,...,1,1,1,1,1,1,1,1,2,2


In [17]:
# clean us cases and deaths
def us_col_clean(case_df, death_df):
    cols_to_drop = ['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Country_Region', 'Lat', 'Long_', 'Combined_Key']
    tmp_case = case_df.drop(cols_to_drop, axis=1)
    tmp_death = death_df.drop(cols_to_drop + ['Population'], axis=1)
    tmp_case['indicator'] = 'Confirmed'
    tmp_death['indicator'] = 'Deaths'
    tmp = pd.concat([tmp_case, tmp_death], axis=0, ignore_index=True)
    return tmp

In [18]:
us_raw = us_col_clean(raw_cases_us, raw_deaths_us)

us_raw.tail()

,Province_State,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20,12/2/20,indicator
6675,Wyoming,0,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,Deaths
6676,Wyoming,0,0,0,0,0,0,0,0,0,...,4,4,4,4,4,4,4,4,4,Deaths
6677,Wyoming,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,9,9,Deaths
6678,Wyoming,0,0,0,0,0,0,0,0,0,...,7,8,8,8,8,8,8,8,8,Deaths
6679,Wyoming,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,2,2,Deaths


In [21]:
def us_shape_clean(df):
    df = pd.melt(df, id_vars=['Province_State', 'indicator'], var_name='Date', value_name='Value')
    #df.head()
    df['Date'] = df['Date'].apply(lambda x: dt.datetime.strptime(str(x), '%m/%d/%y'))
    df = pd.pivot_table(df, index = ['Province_State', 'Date'], columns='indicator', values = 'Value', aggfunc=np.sum).reset_index()
    # Remove non-states
    not_state = ['American Samoa', 'Diamond Princess', 'Grand Princess', 'Guam', 'Northern Mariana Islands', 'Puerto Rico', 'Virgin Islands']
    df = df[~df['Province_State'].isin(not_state)]
    df = df.sort_values(by=['Date', 'Province_State'])

    return df

In [22]:
us_clean = us_shape_clean(us_raw)

In [97]:
us_clean.tail()

indicator,Province_State,Date,Confirmed,Deaths
17009,Virginia,2020-12-01,240063,4093
17324,Washington,2020-12-01,167216,2805
17639,West Virginia,2020-12-01,48818,758
17954,Wisconsin,2020-12-01,416365,3611
18269,Wyoming,2020-12-01,33805,239


In [23]:
def calculate_daily(df, state):
    df = df.sort_values(by=['Date'])
    confirmed = df[df['Province_State'] == state]['Confirmed'].values.tolist()
    deaths = df[df['Province_State'] == state]['Deaths'].values.tolist()
    tmp_df = df[df['Province_State'] == state]
    new_confirmed = [confirmed[0]]
    new_deaths = [deaths[0]]
    for i in range(len(tmp_df)-1):
        new_confirmed.append(confirmed[i+1]-confirmed[i])
        new_deaths.append(deaths[i+1]-deaths[i])
    tmp_df['new_confirmed'] = new_confirmed
    tmp_df['new_deaths'] = new_deaths

    return tmp_df # returns subsetted df with daily new confirmed and daily new deaths

In [24]:
states = sorted(set(us_clean['Province_State'].values.tolist()))
us_final = pd.DataFrame()

for state in states:
    tmp = calculate_daily(us_clean, state=state)
    us_final = us_final.append(tmp, ignore_index=True)

<ipython-input-23-0797f2451e3d>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_df['new_confirmed'] = new_confirmed
<ipython-input-23-0797f2451e3d>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_df['new_deaths'] = new_deaths


In [25]:
us_final.tail()

indicator,Province_State,Date,Confirmed,Deaths,new_confirmed,new_deaths
16111,Wyoming,2020-11-28,31928,215,155,0
16112,Wyoming,2020-11-29,32489,215,561,0
16113,Wyoming,2020-11-30,33305,215,816,0
16114,Wyoming,2020-12-01,33805,239,500,24
16115,Wyoming,2020-12-02,34491,239,686,0


In [26]:
# Compute rolling 7 day average for new_confirmed and new_death for each state
us_final['new_confirmed_7'] = us_final.groupby('Province_State')['new_confirmed'].rolling(7).mean().reset_index(0, drop=True)
us_final['new_deaths_7'] = us_final.groupby('Province_State')['new_deaths'].rolling(7).mean().reset_index(0, drop=True)

# Comptue rolling 14 day avg for new_confirmed and new_deaths for each state
us_final['new_confirmed_14'] = us_final.groupby('Province_State')['new_confirmed'].rolling(14).mean().reset_index(0, drop=True)
us_final['new_deaths_14'] = us_final.groupby('Province_State')['new_deaths'].rolling(14).mean().reset_index(0, drop=True)

In [27]:
us_final.tail(10)

indicator,Province_State,Date,Confirmed,Deaths,new_confirmed,new_deaths,new_confirmed_7,new_deaths_7,new_confirmed_14,new_deaths_14
16106,Wyoming,2020-11-23,29431,202,1262,26,891.142857,8.285714,815.785714,6.285714
16107,Wyoming,2020-11-24,29959,202,528,0,786.571429,6.714286,765.500000,5.357143
16108,Wyoming,2020-11-25,30761,215,802,13,783.714286,8.571429,813.357143,6.285714
16109,Wyoming,2020-11-26,30761,215,0,0,656.000000,5.571429,734.428571,6.285714
16110,Wyoming,2020-11-27,31773,215,1012,0,663.428571,5.571429,745.142857,6.285714
16111,Wyoming,2020-11-28,31928,215,155,0,645.428571,5.571429,717.642857,5.071429
16112,Wyoming,2020-11-29,32489,215,561,0,617.142857,5.571429,713.928571,5.071429
16113,Wyoming,2020-11-30,33305,215,816,0,553.428571,1.857143,722.285714,5.071429
16114,Wyoming,2020-12-01,33805,239,500,24,549.428571,5.285714,668.000000,6.000000
16115,Wyoming,2020-12-02,34491,239,686,0,532.857143,3.428571,658.285714,6.000000


# Step 4: Merging

In [90]:
# ACS population with oxford data

merge1 = pd.merge(oxford, pop, left_on = 'RegionName', right_on='StateName', how='left')
merge1 = merge1.drop(['RegionName'], axis=1)
merge1.tail()

,RegionCode,Date,stringency,containment,StateName,population
17571,WY,2020-11-29,42.59,52.78,Wyoming,578759.0
17572,WY,2020-11-30,42.59,52.78,Wyoming,578759.0
17573,WY,2020-12-01,42.59,52.78,Wyoming,578759.0
17574,WY,2020-12-02,42.59,52.78,Wyoming,578759.0
17575,WY,2020-12-03,42.59,52.78,Wyoming,578759.0


In [91]:
len(oxford) == len(merge1) # sanity check

True

In [92]:
# Merge oxford - StateName, Date & us_final - Province_State, Date

merge2 = pd.merge(us_final, merge1, left_on=['Province_State', 'Date'], right_on=['StateName', 'Date'], how='left')
merge2.tail()

,Province_State,Date,Confirmed,Deaths,new_confirmed,new_deaths,new_confirmed_7,new_deaths_7,new_confirmed_14,new_deaths_14,RegionCode,stringency,containment,StateName,population
16111,Wyoming,2020-11-28,31928,215,155,0,645.428571,5.571429,717.642857,5.071429,WY,42.59,52.78,Wyoming,578759.0
16112,Wyoming,2020-11-29,32489,215,561,0,617.142857,5.571429,713.928571,5.071429,WY,42.59,52.78,Wyoming,578759.0
16113,Wyoming,2020-11-30,33305,215,816,0,553.428571,1.857143,722.285714,5.071429,WY,42.59,52.78,Wyoming,578759.0
16114,Wyoming,2020-12-01,33805,239,500,24,549.428571,5.285714,668.000000,6.000000,WY,42.59,52.78,Wyoming,578759.0
16115,Wyoming,2020-12-02,34491,239,686,0,532.857143,3.428571,658.285714,6.000000,WY,42.59,52.78,Wyoming,578759.0


In [93]:
len(merge2) == len(us_final)

True

In [94]:
merge2.head()

,Province_State,Date,Confirmed,Deaths,new_confirmed,new_deaths,new_confirmed_7,new_deaths_7,new_confirmed_14,new_deaths_14,RegionCode,stringency,containment,StateName,population
0,Alabama,2020-01-22,0,0,0,0,NaN,NaN,NaN,NaN,AL,0.0,0.0,Alabama,4903185.0
1,Alabama,2020-01-23,0,0,0,0,NaN,NaN,NaN,NaN,AL,0.0,0.0,Alabama,4903185.0
2,Alabama,2020-01-24,0,0,0,0,NaN,NaN,NaN,NaN,AL,0.0,0.0,Alabama,4903185.0
3,Alabama,2020-01-25,0,0,0,0,NaN,NaN,NaN,NaN,AL,0.0,0.0,Alabama,4903185.0
4,Alabama,2020-01-26,0,0,0,0,NaN,NaN,NaN,NaN,AL,0.0,0.0,Alabama,4903185.0


In [28]:
merge2.head(20)

,Province_State,Date,Confirmed,Deaths,new_confirmed,new_deaths,new_confirmed_7,new_deaths_7,new_confirmed_14,new_deaths_14,RegionCode,stringency,containment,State_FIPS,StateName,population
0,Alabama,2020-01-22,0,0,0,0,NaN,NaN,NaN,NaN,AL,0.0,0.0,01,Alabama,4864680.0
1,Alabama,2020-01-23,0,0,0,0,NaN,NaN,NaN,NaN,AL,0.0,0.0,01,Alabama,4864680.0
2,Alabama,2020-01-24,0,0,0,0,NaN,NaN,NaN,NaN,AL,0.0,0.0,01,Alabama,4864680.0
3,Alabama,2020-01-25,0,0,0,0,NaN,NaN,NaN,NaN,AL,0.0,0.0,01,Alabama,4864680.0
4,Alabama,2020-01-26,0,0,0,0,NaN,NaN,NaN,NaN,AL,0.0,0.0,01,Alabama,4864680.0
5,Alabama,2020-01-27,0,0,0,0,NaN,NaN,NaN,NaN,AL,0.0,0.0,01,Alabama,4864680.0
6,Alabama,2020-01-28,0,0,0,0,0.0,0.0,NaN,NaN,AL,0.0,0.0,01,Alabama,4864680.0
7,Alabama,2020-01-29,0,0,0,0,0.0,0.0,NaN,NaN,AL,0.0,0.0,01,Alabama,4864680.0
8,Alabama,2020-01-30,0,0,0,0,0.0,0.0,NaN,NaN,AL,0.0,0.0,01,Alabama,4864680.0
9,Alabama,2020-01-31,0,0,0,0,0.0,0.0,NaN,NaN,AL,0.0,0.0,01,Alabama,4864680.0


In [95]:
## rolling 7 day cases and deaths, per 100K (100,000)
final = merge2.copy()
final['per_100k_new_confirmed_7'] = (final['new_confirmed_7']/final['population']) * 100000
final['per_100k_new_deaths_7'] = (final['new_deaths_7']/final['population']) * 100000

In [96]:
final.tail(10)

,Province_State,Date,Confirmed,Deaths,new_confirmed,new_deaths,new_confirmed_7,new_deaths_7,new_confirmed_14,new_deaths_14,RegionCode,stringency,containment,StateName,population,per_100k_new_confirmed_7,per_100k_new_deaths_7
16106,Wyoming,2020-11-23,29431,202,1262,26,891.142857,8.285714,815.785714,6.285714,WY,42.59,52.78,Wyoming,578759.0,153.974773,1.431635
16107,Wyoming,2020-11-24,29959,202,528,0,786.571429,6.714286,765.500000,5.357143,WY,42.59,52.78,Wyoming,578759.0,135.906557,1.160118
16108,Wyoming,2020-11-25,30761,215,802,13,783.714286,8.571429,813.357143,6.285714,WY,42.59,52.78,Wyoming,578759.0,135.412890,1.481001
16109,Wyoming,2020-11-26,30761,215,0,0,656.000000,5.571429,734.428571,6.285714,WY,42.59,52.78,Wyoming,578759.0,113.345970,0.962651
16110,Wyoming,2020-11-27,31773,215,1012,0,663.428571,5.571429,745.142857,6.285714,WY,42.59,52.78,Wyoming,578759.0,114.629504,0.962651
16111,Wyoming,2020-11-28,31928,215,155,0,645.428571,5.571429,717.642857,5.071429,WY,42.59,52.78,Wyoming,578759.0,111.519401,0.962651
16112,Wyoming,2020-11-29,32489,215,561,0,617.142857,5.571429,713.928571,5.071429,WY,42.59,52.78,Wyoming,578759.0,106.632097,0.962651
16113,Wyoming,2020-11-30,33305,215,816,0,553.428571,1.857143,722.285714,5.071429,WY,42.59,52.78,Wyoming,578759.0,95.623320,0.320884
16114,Wyoming,2020-12-01,33805,239,500,24,549.428571,5.285714,668.000000,6.000000,WY,42.59,52.78,Wyoming,578759.0,94.932186,0.913284
16115,Wyoming,2020-12-02,34491,239,686,0,532.857143,3.428571,658.285714,6.000000,WY,42.59,52.78,Wyoming,578759.0,92.068917,0.592401


# Step 5: Mobility Data Cleaning

In [98]:
#Extract csv from zip file
import zipfile
import requests
from io import BytesIO

url = "https://www.gstatic.com/covid19/mobility/Region_Mobility_Report_CSVs.zip"
filename = requests.get(url).content
zf = zipfile.ZipFile(BytesIO(filename), 'r' )

mobility = pd.read_csv(zf.open('2020_US_Region_Mobility_Report.csv'))
mobility.tail(10)

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
727709,US,United States,Wyoming,Weston County,NaN,NaN,56045.0,2020-11-16,NaN,NaN,NaN,NaN,-22.0,NaN
727710,US,United States,Wyoming,Weston County,NaN,NaN,56045.0,2020-11-17,NaN,NaN,NaN,NaN,-17.0,NaN
727711,US,United States,Wyoming,Weston County,NaN,NaN,56045.0,2020-11-18,NaN,NaN,NaN,NaN,-23.0,NaN
727712,US,United States,Wyoming,Weston County,NaN,NaN,56045.0,2020-11-19,NaN,NaN,NaN,NaN,-19.0,NaN
727713,US,United States,Wyoming,Weston County,NaN,NaN,56045.0,2020-11-20,NaN,NaN,NaN,NaN,-18.0,NaN
727714,US,United States,Wyoming,Weston County,NaN,NaN,56045.0,2020-11-23,NaN,NaN,NaN,NaN,-23.0,NaN
727715,US,United States,Wyoming,Weston County,NaN,NaN,56045.0,2020-11-24,NaN,NaN,NaN,NaN,-21.0,NaN
727716,US,United States,Wyoming,Weston County,NaN,NaN,56045.0,2020-11-25,NaN,NaN,NaN,NaN,-27.0,NaN
727717,US,United States,Wyoming,Weston County,NaN,NaN,56045.0,2020-11-26,NaN,NaN,NaN,NaN,-73.0,NaN
727718,US,United States,Wyoming,Weston County,NaN,NaN,56045.0,2020-11-27,NaN,NaN,NaN,NaN,-58.0,NaN


In [99]:
#Calculate mean change
mobility_US = mobility[mobility['country_region_code'] == 'US']
cols = ['date', 'sub_region_1','retail_and_recreation_percent_change_from_baseline', 'grocery_and_pharmacy_percent_change_from_baseline',
      'parks_percent_change_from_baseline', 'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline', 'residential_percent_change_from_baseline']
mobility_grp = mobility_US.groupby(['sub_region_1', 'date'], as_index = False).mean()[cols]
mobility_grp.head(10)

,date,sub_region_1,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,2020-02-15,Alabama,5.155172,0.230769,49.571429,5.761905,0.784314,-0.863636
1,2020-02-16,Alabama,-1.454545,-4.015625,-5.476190,2.136364,-1.920000,0.523810
2,2020-02-17,Alabama,-1.169492,-4.476190,12.933333,5.727273,-16.264706,4.333333
3,2020-02-18,Alabama,-4.280702,-5.000000,-12.466667,-0.681818,0.338235,1.435897
4,2020-02-19,Alabama,1.438596,-0.730159,6.666667,3.714286,1.397059,0.000000
5,2020-02-20,Alabama,-7.568966,-6.687500,-27.588235,1.363636,-0.676471,2.923077
6,2020-02-21,Alabama,0.523810,-2.800000,9.333333,6.136364,1.441176,0.315789
7,2020-02-22,Alabama,4.913793,0.784615,61.583333,10.318182,4.764706,-1.454545
8,2020-02-23,Alabama,3.145455,0.349206,12.055556,10.818182,1.060000,-0.333333
9,2020-02-24,Alabama,-3.684211,-5.854839,-19.600000,-1.500000,0.617647,2.153846


In [103]:
#Rename columns
mobility_grp.columns = ["Date", "Province_State", "retail_recreation", "grocery_pharmacy", "parks",
                                  "transit", "work", "residential"]
mobility_grp['Date'] = mobility_grp['Date'].str.replace("-","")
mobility_grp['Date'] = mobility_grp['Date'].apply(lambda x: dt.datetime.strptime(str(x), '%Y%m%d'))
mobility_grp.head(10)

,Date,Province_State,retail_recreation,grocery_pharmacy,parks,transit,work,residential
0,2020-02-15,Alabama,5.155172,0.230769,49.571429,5.761905,0.784314,-0.863636
1,2020-02-16,Alabama,-1.454545,-4.015625,-5.476190,2.136364,-1.920000,0.523810
2,2020-02-17,Alabama,-1.169492,-4.476190,12.933333,5.727273,-16.264706,4.333333
3,2020-02-18,Alabama,-4.280702,-5.000000,-12.466667,-0.681818,0.338235,1.435897
4,2020-02-19,Alabama,1.438596,-0.730159,6.666667,3.714286,1.397059,0.000000
5,2020-02-20,Alabama,-7.568966,-6.687500,-27.588235,1.363636,-0.676471,2.923077
6,2020-02-21,Alabama,0.523810,-2.800000,9.333333,6.136364,1.441176,0.315789
7,2020-02-22,Alabama,4.913793,0.784615,61.583333,10.318182,4.764706,-1.454545
8,2020-02-23,Alabama,3.145455,0.349206,12.055556,10.818182,1.060000,-0.333333
9,2020-02-24,Alabama,-3.684211,-5.854839,-19.600000,-1.500000,0.617647,2.153846


In [108]:
merged_w_mobility = pd.merge(left=final, right=mobility_grp, how='left', left_on=['Date', 'Province_State'], right_on=['Date', 'Province_State'])
merged_w_mobility.tail(10)

,Province_State,Date,Confirmed,Deaths,new_confirmed,new_deaths,new_confirmed_7,new_deaths_7,new_confirmed_14,new_deaths_14,...,StateName,population,per_100k_new_confirmed_7,per_100k_new_deaths_7,retail_recreation,grocery_pharmacy,parks,transit,work,residential
16106,Wyoming,2020-11-23,29431,202,1262,26,891.142857,8.285714,815.785714,6.285714,...,Wyoming,578759.0,153.974773,1.431635,-13.300000,4.166667,-2.50,-4.444444,-19.818182,5.9
16107,Wyoming,2020-11-24,29959,202,528,0,786.571429,6.714286,765.500000,5.357143,...,Wyoming,578759.0,135.906557,1.160118,-10.000000,9.428571,-10.75,-5.333333,-21.238095,6.4
16108,Wyoming,2020-11-25,30761,215,802,13,783.714286,8.571429,813.357143,6.285714,...,Wyoming,578759.0,135.412890,1.481001,-0.363636,28.500000,-12.00,-10.444444,-35.318182,7.8
16109,Wyoming,2020-11-26,30761,215,0,0,656.000000,5.571429,734.428571,6.285714,...,Wyoming,578759.0,113.345970,0.962651,-78.272727,-36.833333,-32.80,-28.555556,-79.200000,22.5
16110,Wyoming,2020-11-27,31773,215,1012,0,663.428571,5.571429,745.142857,6.285714,...,Wyoming,578759.0,114.629504,0.962651,-24.000000,-21.000000,-7.25,-22.300000,-54.904762,16.4
16111,Wyoming,2020-11-28,31928,215,155,0,645.428571,5.571429,717.642857,5.071429,...,Wyoming,578759.0,111.519401,0.962651,-22.090909,-16.400000,14.20,-13.555556,-17.466667,5.4
16112,Wyoming,2020-11-29,32489,215,561,0,617.142857,5.571429,713.928571,5.071429,...,Wyoming,578759.0,106.632097,0.962651,-20.600000,-18.000000,-14.60,-8.555556,-14.692308,4.2
16113,Wyoming,2020-11-30,33305,215,816,0,553.428571,1.857143,722.285714,5.071429,...,Wyoming,578759.0,95.623320,0.320884,NaN,NaN,NaN,NaN,NaN,NaN
16114,Wyoming,2020-12-01,33805,239,500,24,549.428571,5.285714,668.000000,6.000000,...,Wyoming,578759.0,94.932186,0.913284,NaN,NaN,NaN,NaN,NaN,NaN
16115,Wyoming,2020-12-02,34491,239,686,0,532.857143,3.428571,658.285714,6.000000,...,Wyoming,578759.0,92.068917,0.592401,NaN,NaN,NaN,NaN,NaN,NaN


# Step 6: Final File

In [109]:
file_name = 'us_state_level_clean_' + str(max(merged_w_mobility['Date'].dt.date)) +'.csv'

In [110]:
file_name

'us_state_level_clean_2020-12-02.csv'

In [111]:
merged_w_mobility.to_csv(file_name) # export